In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph


# 도구(Tool) 사용하기

In [2]:
from langchain_teddynote.tools.tavily import TavilySearch

# 검색 도구 생성
tool = TavilySearch(max_results=3)

# 도구 목록에 추가
tools = [tool]

# 도구 실행
tool.invoke("테디노트 랭체인 튜토리얼")

[{'title': 'linktr.ee/teddynote | Linktree',
  'url': 'https://linktr.ee/teddynote',
  'content': '03/04 LangGraph Hands On 튜토리얼 (2시간 분량) [FastCampus] 테디노트의 RAG 비법노트🙌. 🔥[100% 무료] 테디노트 YouTube 콘텐츠 학습 순서🔥. 📘 랭체인 한국어 튜토리얼🇰🇷 ... Github. 9/21 테디노트-Gencon2024-ModularRAG-20240921.pdf.',
  'score': 0.71948266,
  'raw_content': None},
 {'title': ' - LangChain 한국어 튜토리얼 - WikiDocs',
  'url': 'https://wikidocs.net/book/14314',
  'content': "대화내용을 기억하는 RAG 체인 CH13 LangChain Expression Language(LCEL) 01. 구조화된 출력 체인(with_structered_output) CH15 평가(Evaluations) 01. 온라인 평가를 활용한 평가 자동화 CH16 에이전트(Agent) 01. 도구를 활용한 토론 에이전트(Two Agent Debates with Tools) CH17 LangGraph 01. 한글 형태소 분석기(Kiwi, Kkma, Okt) + BM25 검색기 - shcheon99@naver.com, Jan. 9, 2025, 12:28 p.m. 출력된 결과를 비교했을 때, kiwi tokenizer을 사용한 결과와 kkma, okt 를 사용한 결과가 큰 차이가 없다고 봐도 되는 건가요? CH01 LangChain 시작하기 - NamHyeon, Dec. 8, 2024, 1:17 p.m. 좋은 자료를 무료로 공유해 주셔서, 감사한 마음에 '테디노트의 RAG 비법노트' 강의 등록했습니다 ! 대화 토큰 버퍼 메모리(ConversationTokenBufferMemory) - Jan. 16, 202

In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages


# State 정의
class State(TypedDict):
    # list 타입에 add_messages 적용(list 에 message 추가)
    messages: Annotated[list, add_messages]

In [4]:
from langchain_openai import ChatOpenAI

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# LLM 에 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

In [5]:
# 노드 함수 정의
def chatbot(state: State):
    answer = llm_with_tools.invoke(state["messages"])
    # 메시지 목록 반환
    return {"messages": [answer]}  # 자동으로 add_messages 적용

In [6]:
from langgraph.graph import StateGraph

# 상태 그래프 초기화
graph_builder = StateGraph(State)

# 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드(Tool Node)

In [7]:
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    """Run tools requestd in the last AImessage Noede"""

    def __init__(self, tools: list) -> None:
        # 도구 리스트
        self.tools_list = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        # 메시지가 존재할 경우 가장 최근 메시지 1개 추출
        if message := inputs.get("messages", []):
            message = message[-1]
        else:
            raise ValueError("No message found in input")

        # 도구 호출 결과
        outputs = []
        for tool_call in message.tool_calls:
            # 도구 호출 결과 저장
            tool_result = self.tools_list[tool_call["name"]].invoke(tool_call["args"])
            outputs.append(
                ToolMessage(
                    content=json.dumps(
                        tool_result, ensure_ascii=False
                    ), # 도구 호출 결과를 문자열로 변환
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}

# 도구 노드 생성
tool_node = BasicToolNode(tools=[tool])

# 그래프에 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지(Conditional Edge)

In [10]:
from langgraph.graph import START, END

def route_tools(state: State):
    if messages := state.get("messages", []):
        # 가장 최근 AI 메시지를 추출
        ai_message = messages[-1]
    else:
        # 입력 상태에 메시지가 없는 경우 예외 발생
        raise ValueError(f"No message found in input state to tool_edge: {state}")

    # AI 메시지에 도구 호출이 있는 경우 "tools" 반환
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        # 도구 호출이 있는 경우 "tools" 반환
        return "tools"
    # 도구 호출이 없는 경우 "END" 반환
    return END

# 'tools_condition' 함수는 챗봇이 도구 사용을 요청하면 "tools"를 반환하고, 직접 응답이 가능항 경우 "END"를 반환
graph_builder.add_conditional_edges(
    source="chatbot",
    path=route_tools,
    # route_tools의 반환값이 "tools"인 경우 "tools" 노드로, 그렇지 않으면 END 노드로 라우팅
    path_map={"tools": "tools", END: END},
)

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# 그래프 컴파일
graph = graph_builder.compile()

In [12]:
graph.get_graph().print_ascii()

        +-----------+         
        | __start__ |         
        +-----------+         
              *               
              *               
              *               
        +---------+           
        | chatbot |           
        +---------+           
         .         .          
       ..           ..        
      .               .       
+-------+         +---------+ 
| tools |         | __end__ | 
+-------+         +---------+ 


In [13]:
from langchain_teddynote.messages import display_message_tree

question = "테디노트 YouTube"

for event in graph.stream({"messages": [("user", question)]}):
    for key, value in event.items():
        print(f"\n==============\nSTEP: {key}\n==============\n")
        display_message_tree(value["messages"][-1])


STEP: chatbot

    content: ""
    additional_kwargs:
        tool_calls:
            index [0]
                id: "call_594kyYHfqOukQuHniHCoQW02"
                function: {"arguments": "{"query":"테디노트 YouTube"}", "name": "tavily_web_search"}
                type: "function"
        refusal: None
    response_metadata:
        token_usage:
            completion_tokens: 23
            prompt_tokens: 97
            total_tokens: 120
            completion_tokens_details: {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}
            prompt_tokens_details: {"audio_tokens": 0, "cached_tokens": 0}
        model_name: "gpt-4o-mini-2024-07-18"
        system_fingerprint: "fp_dbaca60df0"
        id: "chatcmpl-BPNNa2fdHwdxLpOW73Q5JinwZuiUK"
        finish_reason: "tool_calls"
        logprobs: None
    type: "ai"
    name: None
    id: "run-fff1cac2-2fa4-48bf-a81a-8487b231d3da-0"
    example: False
    tool_calls:
        index [0]
 